In [1]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
import numpy as np
import copy
#import torch
#import torch.nn as nn
from sklearn.metrics import auc, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [7]:
dirs = ["512_runs", "2k_runs", "6k_runs", "6k_512_runs", "6k_2k_runs"]   
image_size = ["Standard", "CHIPS-Medium", "CHIPS-Large", "CHIPS-Large Downsampled\nto Standard", "CHIPS-Large Downsampled\nto CHIPS-Medium"]
p_values = ["p_0"]
fprs
plt.rc('font',family='Times New Roman')

for p_value in p_values:
    for i, dir in enumerate(dirs):
        fpr = np.insert(np.load(f"{p_value}/{dir}/overall_macro_fpr.npy"), 0, 0)
        tpr = np.insert(np.load(f"{p_value}/{dir}/overall_macro_tpr.npy"), 0, 0)
        plt.plot(fpr, tpr, label = f"{image_size[i]}", linewidth=2)

    plt.legend(loc="lower right", prop={'size': 6})
    ax = plt.gca()
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
    plt.savefig(f"pat_auc_macro_{p_value}.pdf", format="pdf", bbox_inches="tight")
    plt.clf()
    
    for i, dir in enumerate(dirs):
        fpr = np.insert(np.load(f"{p_value}/{dir}/overall_micro_fpr.npy"), 0, 0)
        tpr = np.insert(np.load(f"{p_value}/{dir}/overall_micro_tpr.npy"), 0, 0)
        plt.plot(fpr, tpr, label = f"{image_size[i]}", linewidth=2)

    plt.legend(loc="lower right", prop={'size': 6})
    ax = plt.gca()
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
    plt.savefig(f"pat_auc_micro_{p_value}.pdf", format="pdf", bbox_inches="tight")
    plt.clf()

<Figure size 432x288 with 0 Axes>

Code to just get the legend:

In [10]:
dirs = ["512_runs", "2k_runs", "6k_runs", "6k_512_runs", "6k_2k_runs"]   
image_size = ["Standard", "CHIPS", "CHIPS-Large", "CHIPS-Medium Downsampled\nto Standard", "CHIPS-Large Downsampled\nto Standard"]
p_values = ["p_0", "p_10", "p_25"]
plt.rc('font',family='Times New Roman')

for p_value in p_values:
    for i, dir in enumerate(dirs):
        fpr = np.insert(np.load(f"{p_value}/{dir}/overall_macro_fpr.npy"), 0, 0)
        tpr = np.insert(np.load(f"{p_value}/{dir}/overall_macro_tpr.npy"), 0, 0)
        plt.plot(fpr, tpr, label = f"{image_size[i]}", linewidth=2)

    plt.legend(loc="lower right", prop={'size': 6})
    ax = plt.gca()
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
    plt.savefig(f"legend.pdf", format="pdf", bbox_inches="tight")
    plt.clf()

<Figure size 432x288 with 0 Axes>

Code for bootstrap method

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc

def bootstrap_auc(y_true, y_pred, n_bootstraps=1000):
    """Computes the bootstrap AUC score.
      Args:
        y_true: The ground truth labels.
        y_pred: The predicted probabilities.
        n_bootstraps: The number of bootstraps to use.

      Returns:
        The bootstrap AUC score.
      """

    auc_scores = []
    for i in range(n_bootstraps):
        # Sample with replacement.
        idx = np.random.choice(len(y_true), len(y_true))
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        # Compute the AUC score for the bootstrap sample.
        fpr, tpr, thresholds = roc_curve(y_true_bootstrap, y_pred_bootstrap)
        auc_score = auc(fpr, tpr)

        auc_scores.append(auc_score)

    # Compute the mean and standard deviation of the bootstrap AUC scores.
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)

    return mean_auc, std_auc

# Example usage:

y_true = np.array([0, 0, 1, 1])
y_pred = np.array([0.1, 0.4, 0.6, 0.9])

mean_auc, std_auc = bootstrap_auc(y_true, y_pred)

print("Mean AUC:", mean_auc)
print("Standard deviation:", std_auc)